<a href="https://colab.research.google.com/github/CakeNuthep/Create_Knowledge_Graph/blob/main/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/KnowledgeGraph/api-namematching.zip -d /content/

Archive:  /content/drive/MyDrive/KnowledgeGraph/api-namematching.zip
   creating: /content/api-namematching/
   creating: /content/api-namematching/.vscode/
  inflating: /content/api-namematching/.vscode/settings.json  
  inflating: /content/api-namematching/example.py  
   creating: /content/api-namematching/name_matching/
  inflating: /content/api-namematching/name_matching/hamming.py  
  inflating: /content/api-namematching/name_matching/oneway_mymapping.py  
  inflating: /content/api-namematching/name_matching/rule_all.csv  
  inflating: /content/api-namematching/name_matching/__init__.py  
   creating: /content/api-namematching/name_matching/__pycache__/
  inflating: /content/api-namematching/name_matching/__pycache__/hamming.cpython-37.pyc  
  inflating: /content/api-namematching/name_matching/__pycache__/hamming.cpython-38.pyc  
  inflating: /content/api-namematching/name_matching/__pycache__/input.cpython-37.pyc  
  inflating: /content/api-namematching/name_matching/__pycache__

In [5]:
!pip install pythainlp[thai2rom]

     |████████████████████████████████| 11.0MB 5.8MB/s 
     |████████████████████████████████| 747kB 45.6MB/s 


In [228]:
threshold = 0.9
raw_column_name = 'new_name'
raw_column_id = 'id'

target_column_name = 'authFullName'
target_column_id = 'auth_ID'
target_column_source = 'source'



In [229]:
import pandas as pd
import json
from name_matching import name_matching

In [241]:
def readJson(list_path,list_source,list_sample,random_state=4):
  li = []
  for index in range(len(list_path)):
      sample = list_sample[index]
      filename = list_path[index]
      df = pd.read_csv(filename, index_col=None, header=0)
      df = df.rename(columns={raw_column_id: target_column_id,raw_column_name: target_column_name})
      # df = df.set_index(target_column_id)
      print(df)
      print(f'sample:{sample}')
      df = df.sample(n=sample,random_state=random_state,replace=False)
      df[target_column_source] = list_source[index]
      li.append(df[[target_column_id,target_column_name,target_column_source]])
  df_all = pd.concat(li, axis=0, ignore_index=True)
  return df_all

In [269]:
def separateFirstNameLastName(text):
  first_name = ' '
  last_name = ' '
  text_split = text.split()
  first_name = text_split[0]
  if len(text_split) > 1:
    last_name = ' '.join(text_split[1:])
  return first_name, last_name

def similarity(text1,text2):
  first_name1, last_name1 = separateFirstNameLastName(text1)
  first_name2, last_name2 = separateFirstNameLastName(text2)
  if first_name1[0] == first_name2[0] and last_name1[0] == last_name2[0]:
    result = name_matching(text1,text2,limit = threshold ,score = 1)
  else:
    result = 0
  return result

def similarity_dataframe(df,authID,text,source):
  list_similarity = []
  for index, row in df_all.iterrows():
    temp_name = row[target_column_name]
    temp_id = row[target_column_id]
    temp_source = row[target_column_source]
    if temp_id == authID:
      continue
    result_similarity = similarity(text,temp_name)
    if result_similarity > threshold:
      list_similarity.append({
        target_column_id: temp_id,
        target_column_name: temp_name,
        target_column_source: temp_source,
        "similarity": result_similarity
      })
  json_similarity = {target_column_id:authID,'name':text,target_column_source:source,'list_similarity':list_similarity}
  return json_similarity

def crate_json_similarity(df,start_index,end_index,step_size=1000):
  if end_index > len(df):
    end_index = len(df)
  for index in range(start_index, end_index, step_size):
    last_index = index+step_size
    if last_index > end_index:
      last_index = end_index
    print(f"process: index {index} to {last_index}")
    series_similarities = df[index:last_index].apply(lambda x: similarity_dataframe(df, x[target_column_id],x[target_column_name],x[target_column_source]), axis=1)
    with open('df.json', 'w', encoding='utf-8') as file:
      series_similarities.to_json(f"similarity/similarity_{index}_{last_index}.json",orient='records', force_ascii=False)


# TH-TH Similarity

In [243]:
list_source = ['tnrr','tci-thaijo','royin','nrms','adc']
list_path_author_th = ['/content/drive/MyDrive/KnowledgeGraph/Integrated/tnrr/author_thai.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/tci-thaijo/thaijo_author_thai.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/royin/royin_author_thai.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/nrms/nrms_author_thai.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/adct/adct_author_thai.csv']


In [244]:
list_sample = [3109,3109,219,3110,453]
random_state = 4

In [245]:
df_all = readJson(list_path_author_th,list_source,list_sample,random_state)

        Unnamed: 0  auth_ID            authFullName
0                0        0         อภิรัช สกุลณียา
1                1        1       เกสรา ศรีพิชญาการ
2                2        2           แสงหล้า พลนอก
3                3        3         เชาวนี ล่องชูผล
4                4        4  สัญฉกร วุฒิสิทธิกุลกิจ
...            ...      ...                     ...
261658      410479   410479         อภิษฎา อรัณยภาค
261659      410483   410483      สรินพร แสงประดิษฐ์
261660      410484   410484        นันทนา ศิริวัฒน์
261661      410485   410485      เปรมฤดี สุกฤตานนท์
261662      410487   410487      มงคล กิตติอารีพงศ์

[261663 rows x 3 columns]
sample:3109
       Unnamed: 0  auth_ID             authFullName
0               0   410489        ปรเมษฐ์ ธันวานนท์
1               1   410490           ชัยกร ยิ่งเสรี
2               2   410491           วรพล พงษ์เพ็ชร
3               3   410492         ธนภัทร ฆังคะจิตร
4               4   410493           ปฤณ เทพนรินทร์
...           ...      ..

In [236]:
df_all

,auth_ID,authFullName,source
0,23282,อภิสิฏฐ์ ศงสะเสน,tnrr
1,372219,พิสิษฐ์ สุขสวัสดิ์,tnrr
2,396881,พิเชษฐ์ ทรงณัฎฐพจน์,tnrr
3,50605,ปริยา ปราณีตพลกรัง,tnrr
4,201891,จรินทรัตน์ กิติศรี,tnrr
...,...,...,...
9995,542493,กันยรัตน์ โหละสุต,adc
9996,542212,สุชาติ แย้มแม่น,adc
9997,542584,สยาม ยิ้มศิริ,adc
9998,542185,ทวีศักดิ์ แตะกระโทก,adc


In [194]:
crate_json_similarity(df_all,0,len(df_all),10000)

process: index 0 to 10000


## read Result TH-TH Json File

In [201]:
import json
import numpy as np

In [198]:
# Opening JSON file
f = open('/content/tnrr/similarity_0_10000.json')
  
# returns JSON object as 
# a dictionary
similarity_json = json.load(f)

In [199]:
similarity_json

[{'auth_ID': 23282,
  'list_similarity': [],
  'name': 'อภิสิฏฐ์ ศงสะเสน',
  'source': 'tnrr'},
 {'auth_ID': 372219,
  'list_similarity': [],
  'name': 'พิสิษฐ์ สุขสวัสดิ์',
  'source': 'tnrr'},
 {'auth_ID': 396881,
  'list_similarity': [],
  'name': 'พิเชษฐ์ ทรงณัฎฐพจน์',
  'source': 'tnrr'},
 {'auth_ID': 50605,
  'list_similarity': [],
  'name': 'ปริยา ปราณีตพลกรัง',
  'source': 'tnrr'},
 {'auth_ID': 201891,
  'list_similarity': [],
  'name': 'จรินทรัตน์ กิติศรี',
  'source': 'tnrr'},
 {'auth_ID': 267337,
  'list_similarity': [],
  'name': 'สรพงค์ ศรียานงค์',
  'source': 'tnrr'},
 {'auth_ID': 258956,
  'list_similarity': [],
  'name': 'เพิ่มทรัพย์ หิรัญสาย',
  'source': 'tnrr'},
 {'auth_ID': 312954,
  'list_similarity': [],
  'name': 'ศิริพร สิงห์หล้า',
  'source': 'tnrr'},
 {'auth_ID': 301255,
  'list_similarity': [],
  'name': 'จตุพร วุฒิกนก',
  'source': 'tnrr'},
 {'auth_ID': 201562,
  'list_similarity': [],
  'name': 'พินทอง ปินใจ',
  'source': 'tnrr'},
 {'auth_ID': 149095,
  'li

In [200]:
len(similarity_json)

10000

In [220]:
def getHasSimilarity(similarity_json,min_person = 1):
  list_has_similarity = []
  for data in similarity_json:
    if len(data['list_similarity']) >= min_person:
      list_has_similarity.append(data)
  return list_has_similarity

In [221]:
list_has_similarity = getHasSimilarity(similarity_json)

In [222]:
len(list_has_similarity)

254

In [223]:
list_has_similarity

[{'auth_ID': 150024,
  'list_similarity': [{'authFullName': 'บุญเกียรติ ชีวะตระกูลกิจ',
    'auth_ID': 503058,
    'similarity': 1.0,
    'source': 'tci-thaijo'}],
  'name': 'บุญเกียรติ ชีวะตระกูลกิจ',
  'source': 'tnrr'},
 {'auth_ID': 320731,
  'list_similarity': [{'authFullName': 'รุ้งทิพย์ เมืองโคตร',
    'auth_ID': 528727,
    'similarity': 1.0,
    'source': 'nrms'}],
  'name': 'รุ้งทิพย์ เมืองโคตร',
  'source': 'tnrr'},
 {'auth_ID': 10368,
  'list_similarity': [{'authFullName': 'พงศ์เดช ไชยคุตร',
    'auth_ID': 540229,
    'similarity': 1.0,
    'source': 'nrms'}],
  'name': 'พงศ์เดช ไชยคุตร',
  'source': 'tnrr'},
 {'auth_ID': 181920,
  'list_similarity': [{'authFullName': 'ภาณุมาศ ลาดปาละ',
    'auth_ID': 539760,
    'similarity': 0.9333333333,
    'source': 'nrms'}],
  'name': 'ภานุมาศ ลาดปาละ',
  'source': 'tnrr'},
 {'auth_ID': 107039,
  'list_similarity': [{'authFullName': 'สรวงสุดา ปานสกุล',
    'auth_ID': 455328,
    'similarity': 1.0,
    'source': 'tci-thaijo'}],
  'name'

In [226]:
list_has_similarity = getHasSimilarity(similarity_json,2)

In [227]:
list_has_similarity

[]

# EN - EN Similarity

In [271]:
list_source = ['tnrr','tci-thaijo','nrms','adc','dblp','dblp-xml','pubmed']
list_path_author_eng = ['/content/drive/MyDrive/KnowledgeGraph/Integrated/tnrr/author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/tci-thaijo/thaijo_author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/nrms/nrms_author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/adct/adct_author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/dblp/dblp_author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/dblp-xml/authorEN/dblp-xml_author_eng.csv',
                       '/content/drive/MyDrive/KnowledgeGraph/Integrated/pubmed/authorEN/pubmed_author_eng.csv']


In [272]:
list_sample = [2500,2500,13,212,2500,2500,2500]
random_state = 4

In [273]:
df_all_eng = readJson(list_path_author_eng,list_source,list_sample,random_state)

        Unnamed: 0  auth_ID                                    authFullName
0               12       12  Chulalongkorn University. Faculty of Economics
1               33       33                            Thawatchai Tuntulani
2               34       34    Chulalongkorn University. Faculty of Science
3               37       37                                     Udom Kokpol
4               38       38                                Tirayut Vilaivan
...            ...      ...                                             ...
148806      410480   410480                              Passanan Assavarak
148807      410481   410481                           Watanyoo Pratakpiriya
148808      410482   410482                               Ashadi Bin Yaacob
148809      410486   410486                                       Alavandar
148810      410488   410488                                         Waewdao

[148811 rows x 3 columns]
sample:2500
       Unnamed: 0  auth_ID                    aut

In [274]:
df_all_eng

,auth_ID,authFullName,source
0,94835,Bradley P.,tnrr
1,42667,Kanoknard Sanganate,tnrr
2,232682,Reunchan,tnrr
3,1487,Prapapan Kongwitseranee,tnrr
4,103763,Sattin,tnrr
...,...,...,...
12720,667736,L L Grechishkin,pubmed
12721,613910,K C Hayes,pubmed
12722,660142,N K Barkov,pubmed
12723,652849,M Veltri,pubmed


In [275]:
# get rid sub text in thai language
def hasThaiWord(str):
    for i in str:
        if i in ('ก','ข','ฃ','ค','ฅ','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ'):
            return True
    else:
        return False

In [276]:
df_all['has_thai'] = df_all['authFullName'].apply(hasThaiWord)

In [277]:
df_all

,auth_ID,authFullName,source,has_thai
0,94835,Bradley P.,tnrr,False
1,42667,Kanoknard Sanganate,tnrr,False
2,232682,Reunchan,tnrr,False
3,1487,Prapapan Kongwitseranee,tnrr,False
4,103763,Sattin,tnrr,False
...,...,...,...,...
12720,40208,L L Grechishkin,pubmed,False
12721,13979,K C Hayes,pubmed,False
12722,22477,N K Barkov,pubmed,False
12723,6590,M Veltri,pubmed,False


In [278]:
df_all = df_all[df_all['has_thai']==False]

In [279]:
df_all = df_all[['auth_ID','authFullName','source']]

In [280]:
df_all

,auth_ID,authFullName,source
0,94835,Bradley P.,tnrr
1,42667,Kanoknard Sanganate,tnrr
2,232682,Reunchan,tnrr
3,1487,Prapapan Kongwitseranee,tnrr
4,103763,Sattin,tnrr
...,...,...,...
12720,40208,L L Grechishkin,pubmed
12721,13979,K C Hayes,pubmed
12722,22477,N K Barkov,pubmed
12723,6590,M Veltri,pubmed


In [281]:
crate_json_similarity(df_all,0,len(df_all),10000)

process: index 0 to 10000
process: index 10000 to 12514


## Read Result EN-EN JSON File

In [282]:
import json
import numpy as np

In [283]:
# Opening JSON file
f = open('/content/similarity/similarity_0_10000.json')
  
# returns JSON object as 
# a dictionary
similarity_json = json.load(f)

In [284]:
similarity_json

[{'auth_ID': 94835,
  'list_similarity': [],
  'name': 'Bradley P.',
  'source': 'tnrr'},
 {'auth_ID': 42667,
  'list_similarity': [],
  'name': 'Kanoknard Sanganate',
  'source': 'tnrr'},
 {'auth_ID': 232682,
  'list_similarity': [],
  'name': 'Reunchan',
  'source': 'tnrr'},
 {'auth_ID': 1487,
  'list_similarity': [],
  'name': 'Prapapan Kongwitseranee',
  'source': 'tnrr'},
 {'auth_ID': 103763,
  'list_similarity': [],
  'name': 'Sattin',
  'source': 'tnrr'},
 {'auth_ID': 260241,
  'list_similarity': [],
  'name': 'Krissana Auynirundronkul',
  'source': 'tnrr'},
 {'auth_ID': 21670, 'list_similarity': [], 'name': 'Leo', 'source': 'tnrr'},
 {'auth_ID': 64738,
  'list_similarity': [],
  'name': 'Sukalaya lerdlum',
  'source': 'tnrr'},
 {'auth_ID': 43716,
  'list_similarity': [],
  'name': 'Songsak Kietchusakul',
  'source': 'tnrr'},
 {'auth_ID': 335137,
  'list_similarity': [],
  'name': 'Sataree Chitanonda',
  'source': 'tnrr'},
 {'auth_ID': 18715,
  'list_similarity': [],
  'name': '

In [285]:
len(similarity_json)

10000

In [286]:
def getHasSimilarity(similarity_json,min_person = 1):
  list_has_similarity = []
  for data in similarity_json:
    if len(data['list_similarity']) >= min_person:
      list_has_similarity.append(data)
  return list_has_similarity

In [287]:
list_has_similarity = getHasSimilarity(similarity_json)

In [288]:
len(list_has_similarity)

31

In [289]:
list_has_similarity

[{'auth_ID': 284305,
  'list_similarity': [{'authFullName': 'Sujit Kumar',
    'auth_ID': 590313,
    'similarity': 1.0,
    'source': 'dblp'}],
  'name': 'Sujit Kumar',
  'source': 'tnrr'},
 {'auth_ID': 24653,
  'list_similarity': [{'authFullName': 'Bangorn Sripanidkulchai',
    'auth_ID': 467737,
    'similarity': 0.9565217391,
    'source': 'tci-thaijo'}],
  'name': 'Bungorn Sripanidkulchai',
  'source': 'tnrr'},
 {'auth_ID': 233343,
  'list_similarity': [{'authFullName': 'Yuttasak Chammui',
    'auth_ID': 416885,
    'similarity': 1.0,
    'source': 'tci-thaijo'}],
  'name': 'Yuttasak Chammui',
  'source': 'tnrr'},
 {'auth_ID': 129721,
  'list_similarity': [{'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 454448,
    'similarity': 0.9230769231,
    'source': 'tci-thaijo'},
   {'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 541941,
    'similarity': 0.9230769231,
    'source': 'adc'}],
  'name': 'Chalie Chareonlarpnopparut',
  'source': 'tnrr'},
 {'auth_ID

In [290]:
list_has_similarity = getHasSimilarity(similarity_json,2)

In [291]:
list_has_similarity

[{'auth_ID': 129721,
  'list_similarity': [{'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 454448,
    'similarity': 0.9230769231,
    'source': 'tci-thaijo'},
   {'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 541941,
    'similarity': 0.9230769231,
    'source': 'adc'}],
  'name': 'Chalie Chareonlarpnopparut',
  'source': 'tnrr'},
 {'auth_ID': 454448,
  'list_similarity': [{'authFullName': 'Chalie Chareonlarpnopparut',
    'auth_ID': 129721,
    'similarity': 0.9230769231,
    'source': 'tnrr'},
   {'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 541941,
    'similarity': 1.0,
    'source': 'adc'}],
  'name': 'Chalie Charoenlarpnopparut',
  'source': 'tci-thaijo'},
 {'auth_ID': 541941,
  'list_similarity': [{'authFullName': 'Chalie Chareonlarpnopparut',
    'auth_ID': 129721,
    'similarity': 0.9230769231,
    'source': 'tnrr'},
   {'authFullName': 'Chalie Charoenlarpnopparut',
    'auth_ID': 454448,
    'similarity': 1.0,
    'source': 'tci-